In [59]:
import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout
from pathlib import Path
from ipywidgets import GridspecLayout
from functools import partial
from IPython.display import display


In [57]:

# ファイルの一覧を取得する
def list_dir(path)->tuple:
    return tuple(
        i.stem+"/" if i.is_dir() else i.stem 
        for i 
        in Path(path).iterdir()
    )

# そのパスの一覧を選択肢にいれたselectを作成する
def generate_select(path):
    return widgets.Select(
        options=list_dir(path),
        disabled=False,
        rows=10
    )

# パスの一覧
paths = [".", ".git", ".git/logs", ".git/logs/refs", ".git/logs/refs/remotes",".git/logs/refs/remotes/origin"]
paths = [Path(pathstr) for pathstr in paths]

# 表示するセレクタの一覧
select_objects = list()
for i, path in enumerate(paths):
    select_objects.append(generate_select(path))

# デバッグ用の出力
output = widgets.Output(layour={'border': '1px solid black'})

# 選択肢が変化した際にその1階層下の選択肢を更新する
@output.capture()
def update_child(changes, root_index): # changesが第一変数で入ってくることに気づかず苦戦した
    if changes["name"] == "value":
        # 今現在選択しているもののパスを取得する
        curdir = paths[root_index] 
        select = select_objects[root_index]
        next_path = curdir / select.get_interact_value()
        if root_index < len(paths) - 1: # 一番右の選択をした場合には変更しない
            # 今現在選択しているもののパスを取得する
            curdir = paths[root_index] 
            select = select_objects[root_index]
            next_path = curdir / select.get_interact_value()
            
            # 変更することによって
            grid[:, root_index+1].unobserve(handlers[root_index+1])
            if next_path.is_dir():
                grid[:, root_index+1].options=list_dir(next_path)
            else:
                grid[:, root_index+1].options=[""]
            grid[:, root_index+1].observe(handlers[root_index+1])
            paths[root_index+1] = next_path
        for i in range(root_index + 2, len(paths)):
            
            # 変更することによって
            grid[:, i].unobserve(handlers[i])
            grid[:, i].options=[""]
            grid[:, i].observe(handlers[i])
            paths[i] = ""

handlers = [partial(update_child, root_index=i) for i in range(len(paths))]

In [60]:
# gridを作成して登録する
grid = GridspecLayout(1, len(paths))
for i, select in enumerate(select_objects):
    grid[:, i] = select
    grid[:, i].observe(handlers[i])
display(grid, output)

GridspecLayout(children=(Select(layout=Layout(grid_area='widget001'), options=('make-gui', 'README', 'Pipfile'…

Output()